In [2]:
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload

In [3]:
#!pip install sentence_transformers

In [4]:
import collections
import concurrent.futures
import datetime
import json
import termcolor

import numpy as np

from IPython import display

from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia import components as generic_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.utils import plotting

from examples.custom_components import behavior

In [5]:
local_models = True

if local_models:
    # Setup LLM
    from concordia.language_model import ollama_model
    model = ollama_model.OllamaLanguageModel(
        model_name='llama2:13b',
        streaming=True
    )

    # Setup measurements and clock
    measurements = measurements_lib.Measurements()
    time_step = datetime.timedelta(minutes=20)
    SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

    START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
    clock = game_clock.MultiIntervalClock(
        start=SETUP_TIME,
        step_sizes=[time_step, datetime.timedelta(seconds=10)])

    # Setup sentence encoder
    from sentence_transformers import SentenceTransformer
    st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
    embedder = st5_model.encode

else:
    from concordia.language_model import gpt_model
    from concordia.language_model import gcloud_model
    # @title Language Model - pick your model and provide keys
    CLOUD_PROJECT_ID = '' #@param {type: 'string'}
    GPT_API_KEY = '' #@param {type: 'string'}
    GPT_MODEL_NAME = '' #@param {type: 'string'}

    USE_CLOUD = True #@param {type: 'boolean'}

    if USE_CLOUD:
        model = gcloud_model.CloudLanguageModel(project_id= CLOUD_PROJECT_ID)
    else:
        model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY, model_name=GPT_MODEL_NAME)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()


In [6]:
# @title Generic memories are memories that all players and GM share.

shared_memories = [
    'Alice returns late from work and finds Dorothy at her doorstep.',
    "Dorothy has to move out of her house tonight and needs someone to help her move.",
]

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)
print(shared_context)

/Users/wil/.pyenv/versions/3.12.1/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Alice returns home late from work to find Dorothy waiting at her doorstep, seeking help with an urgent move as she must vacate her house that night.
Alice returns home late from work to find Dorothy waiting at her doorstep, seeking help with an urgent move as she must vacate her house that night.


In [7]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [8]:
#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes
NUM_PLAYERS = 2

scenario_premise = [

    (
        'Alice returns late from work and finds Dorothy at her doorstep. '
        + 'Dorothy has to move out of her house tonight and needs someone to help her move.'
    ),
]
player_configs = [
    formative_memories.AgentConfig(
        name='Alice',
        gender='female',
        goal='Alice is exhausted after work, and she needs to help her grandmother all day the next day.',
        context=shared_context,
        traits='responsibility: medium; aggression: medium',
    ),
    formative_memories.AgentConfig(
        name='Dorothy',
        gender='female',
        goal=(
            'Dorothy has planned on Bob helping her move, but Bob didn''t show up. '
        ),
        context=shared_context,
        traits='responsibility: medium; aggression: medium',
    ),
]

In [9]:
mem = formative_memory_factory.make_memories(player_configs[0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

   Alice is a responsible and medium-aggressive female character who works as a project manager for a real estate development company. She has been in the industry for over five years and has risen through the ranks due to her exceptional organizational skills and attention to detail.

On a typical day, Alice wakes up early to prepare for work and spends time with her family before heading out the door. She enjoys her job and finds it fulfilling, but she often feels overwhelmed by the demands of her role and the pressure to meet deadlines.

Alice's goals and aspirations include becoming a senior project manager and eventually starting her own real estate development company. She is driven by a desire to create beautiful and functional spaces that improve people's lives. She also hopes to have a family of her own one day.

Despite her success, Alice struggles with self-doubt and fear of failure. She is afraid of not meeting expectations and disappointing her colleagues and clients. She 

In [25]:
%autoreload 2
from examples.custom_components import behavior

behaviors = behavior.BehavioralChoices(
    name='behaviours',
    model=model,
    memory=mem,
    player_config=player_configs[0],
    clock_now=clock.now,
    verbose=False
  )

behaviors.update()

consequences = behavior.BehavioralConsequences(
    name='behaviours',
    model=model,
    memory=mem,
    player_config=player_configs[0],
    components=[behaviors],
    clock_now=clock.now,
    verbose=False
  )

consequences.update()

 1. Apologize to Bob and try to make amends for the damage to his car.
2. Refuse to accept responsibility for the damage and instead blame someone or something else.
3. Try to intimidate or threaten Bob into accepting her version of events.
4. Offer to pay for the damages, but only if Bob agrees to drop the matter.
5. Attempt to bribe Bob with gifts or favors in exchange for his silence.
6. Use her charm and charisma to try to talk her way out of the situation.
7. Become defensive and hostile when confronted by Bob, lashing out at him verbally or physically.
8. Try to manipulate or guilt trip Bob into accepting her version of events.
9. Use her impulsive nature to make rash decisions without thinking through the consequences.
10. Take responsibility for her actions and apologize sincerely to Bob, hoping to make things right.

Note: The above behaviors are not mutually exclusive, and Alice could exhibit a combination of several traits depending on the situation. Also, it is essential to

In [70]:
import re

replacements = [
  (r'^\n*?.*?\n*?\[', '['),
  # (r'\n', ''),
  (r'\n\],\n*?\[\n', ',\n')
]

item = consequences._consequences[9]

for old, new in replacements:
  item = re.sub(old, new, item)
  
item = json.loads(item)
print(item)

[{'consequence': 'Positive consequence', 'description': "Alice's apology and willingness to take responsibility for her actions may help to repair her relationship with Bob and potentially lead to a more positive and productive dynamic between them."}, {'consequence': 'Negative consequence', 'description': 'Bob may still be angry and resentful towards Alice, and her apology may not be enough to make amends for the damage she has caused. He may continue to hold grudges and be less willing to work with her in the future.'}, {'consequence': 'Positive consequence', 'description': 'By taking responsibility for her actions, Alice may gain a sense of personal growth and development, as well as earn the respect of others for her willingness to own up to her mistakes.'}, {'consequence': 'Negative consequence', 'description': "If Bob does not accept Alice's apology or if he continues to hold grudges against her, it could lead to further conflict and tension between them, potentially causing more

In [12]:
test = model.sample_text(
  f'Memories of Alice:{behaviors._memory.retrieve_recent(100)}'
  f'Instructions: \n'
  f'Given the memories above, evaluate the potential consequences '
  f'of Alice engaging in the following behaviour:\n'
  f"Defiant - Alice may challenge Bob's ownership of the car and "
  f"refuse to accept responsibility for the damage, leading to a "
  f"confrontation and potentially straining their relationship. "
  f"List five potential consequences, and include a variety of both "
  f"positive and negative consequences."
)

# Full description of format of how it would answer the question. Be sure to structure
# If it's in the wrong format, make the LM redo it.

# Working memory instead of associative memory - 
# wipe it after each call so we can make sure that it's not getting bleedover from previous prompts



  Based on the memories provided, it is likely that Alice engaging in defiant behavior by challenging Bob's ownership of the car and refusing to accept responsibility for the damage could have the following potential consequences:

1. Confrontation: Alice's defiance could lead to a confrontation with Bob, potentially straining their relationship and causing tension within the group.
2. Damage to Relationship: By refusing to accept responsibility for the damage, Alice may be seen as irresponsible and unreliable by Bob and other members of the group, potentially damaging her relationships with them.
3. Legal Consequences: Depending on the severity of the damage, Alice's actions could lead to legal consequences, such as fines or even criminal charges, which could have long-lasting effects on her life.
4. Loss of Trust: By challenging Bob's ownership of the car and refusing to accept responsibility, Alice may lose the trust of her friends and peers, potentially damaging her reputation and 

In [ ]:
[
  {
  "behavior": "Stuff",
  "consequences": [
    {
      "consequence": "Bad",
      "description": "Bad stuff happens",
      "rating": 2,
      "probability": 0.2,
    },
    {
      "consequence": "Good",
      "description": "Good stuff happens",
      "rating": 8,
      "probability": 0.5
    }
  ],
  "people": [
        {
          "person": "Alice",
          "rating": 7,
          "importance": 1
        },
        {
          "person": "Bob",
          "rating": 1,
          "importance": 8
        }
  ]
}, {
  "behavior": 2,
  "consequences": ...,
  "people": ...,
  }
]

In [13]:
model.sample_text(
  f"Please reformat the following list into a JSON object with "
  f"the following properties:\n"
  f"'behaviour': A string including the behaviour and a description of it.\n"
  f"'consequences': An array including the possible consequences.\n"
  f"{test}"
)


Here is the list reformatted as a JSON object with the specified properties:

{
"behaviour": "Alice engaging in defiant behavior by challenging Bob's ownership of the car and refusing to accept responsibility for the damage",
"consequences": [
{
"consequence": "Confrontation",
"description": "Alice's defiance could lead to a confrontation with Bob, potentially straining their relationship and causing tension within the group."
},
{
"consequence": "Damage to Relationship",
"description": "By refusing to accept responsibility for the damage, Alice may be seen as irresponsible and unreliable by Bob and other members of the group, potentially damaging her relationships with them."
},
{
"consequence": "Legal Consequences",
"description": "Depending on the severity of the damage, Alice's actions could lead to legal consequences, such as fines or even criminal charges, which could have long-lasting effects on her life."
},
{
"consequence": "Loss of Trust",
"description": "By challenging Bob'

'\nHere is the list reformatted as a JSON object with the specified properties:\n\n{\n"behaviour": "Alice engaging in defiant behavior by challenging Bob\'s ownership of the car and refusing to accept responsibility for the damage",\n"consequences": [\n{\n"consequence": "Confrontation",\n"description": "Alice\'s defiance could lead to a confrontation with Bob, potentially straining their relationship and causing tension within the group."\n},\n{\n"consequence": "Damage to Relationship",\n"description": "By refusing to accept responsibility for the damage, Alice may be seen as irresponsible and unreliable by Bob and other members of the group, potentially damaging her relationships with them."\n},\n{\n"consequence": "Legal Consequences",\n"description": "Depending on the severity of the damage, Alice\'s actions could lead to legal consequences, such as fines or even criminal charges, which could have long-lasting effects on her life."\n},\n{\n"consequence": "Loss of Trust",\n"descriptio

In [9]:
from examples.custom_components import behavior

def build_agent(
    agent_config,
    player_names: list[str],
    measurements: measurements_lib.Measurements | None = None,
):
  mem = formative_memory_factory.make_memories(agent_config)

  behaviours = behavior.BehavioralChoices(
    name='behaviours',
    model=model,
    memory=mem,
    agent_name=agent_config.name,
    clock_now=clock.now
  )

  self_perception = components.self_perception.SelfPerception(
      name='self perception',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      verbose=True,
  )
  situation_perception = components.situation_perception.SituationPerception(
      name='situation perception',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      verbose=True,
  )
  person_by_situation = components.person_by_situation.PersonBySituation(
      name='person by situation',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      components=[self_perception, situation_perception],
      verbose=True,
  )
  persona = components.sequential.Sequential(
      name='persona',
      components=[
          self_perception,
          situation_perception,
          person_by_situation,
      ],
  )
  current_time_component = components.report_function.ReportFunction(
      name='current_time', function=clock.current_time_interval_str
  )

  current_obs = components.observation.Observation(
            agent_name=agent_config.name,
      clock_now=clock.now,
      memory=mem,
      timeframe=clock.get_step_size(),
      component_name='current observations',
  )
  summary_obs = components.observation.ObservationSummary(
      agent_name=agent_config.name,
      model=model,
      clock_now=clock.now,
      memory=mem,
      timeframe_delta_from=datetime.timedelta(hours=4),
      timeframe_delta_until=datetime.timedelta(hours=1),
      components=[persona],
      component_name='summary of observations',
  )


  goal_metric = goal_achievement.GoalAchievementMetric(
      model=model,
      player_name=agent_config.name,
      player_goal=agent_config.goal,
      clock=clock,
      name='Goal Achievement',
      measurements=measurements,
      channel='goal_achievement',
      verbose=False,
  )
  morality_metric = common_sense_morality.CommonSenseMoralityMetric(
      model=model,
      player_name=agent_config.name,
      clock=clock,
      name='Morality',
      verbose=False,
      measurements=measurements,
      channel='common_sense_morality',
  )

  agent = basic_agent.BasicAgent(
      model,
      mem,
      agent_name=agent_config.name,
      clock=clock,
      verbose=False,
      components=[
          persona,
          current_time_component,
          summary_obs,
          current_obs,
          goal_metric,
          morality_metric,
      ],
      update_interval=time_step,
  )
  reputation_metric = opinion_of_others.OpinionOfOthersMetric(
      model=model,
      player_name=agent_config.name,
      player_names=player_names,
      context_fn=agent.state,
      clock=clock,
      name='Opinion',
      verbose=False,
      measurements=measurements,
      channel='opinion_of_others',
      question="What is {opining_player}'s opinion of {of_player}?",
  )
  agent.add_component(reputation_metric)

  return agent